In [23]:
import pandas as pd
import os
import json
from concurrent.futures import ThreadPoolExecutor

In [24]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC

from utilities.utilities import element_exists 

In [26]:
chrome_options = ChromeOptions()
service = ChromeService(executable_path=ChromeDriverManager().install())

KeyboardInterrupt: 

In [ ]:
# link where to download data
link = 'https://app.virtuals.io/virtuals/9042/contribution'

In [ ]:
driver = webdriver.Chrome(options=chrome_options, service=service)
driver.get(link)
_ = WebDriverWait(driver, timeout=20).until(lambda driver: driver.execute_script('return document.readyState === "complete"'))

In [ ]:
contribs_btn = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]')
contribs_btn.click()

In [ ]:
element_exists(driver, By.XPATH, '/html/body/path')

NameError: name 'element_exists' is not defined

In [ ]:
# select all div elements first
divs = driver.find_elements(By.CSS_SELECTOR, 'div.MuiStack-root')
[div for div in divs if element_exists(div, By.CSS_SELECTOR, 'span.MuiChip-label')]

# check each div elements have its decendant span.MuiChip-label if any 

[<selenium.webdriver.remote.webelement.WebElement (session="99d230b05e3d95c53ee3771b7e76fad8", element="f.D2E048430D79195F3054D6CB1F377AFB.d.C3CE2A2E765A6A89FC277812E20486A1.e.142")>,
 <selenium.webdriver.remote.webelement.WebElement (session="99d230b05e3d95c53ee3771b7e76fad8", element="f.D2E048430D79195F3054D6CB1F377AFB.d.C3CE2A2E765A6A89FC277812E20486A1.e.215")>,
 <selenium.webdriver.remote.webelement.WebElement (session="99d230b05e3d95c53ee3771b7e76fad8", element="f.D2E048430D79195F3054D6CB1F377AFB.d.C3CE2A2E765A6A89FC277812E20486A1.e.216")>,
 <selenium.webdriver.remote.webelement.WebElement (session="99d230b05e3d95c53ee3771b7e76fad8", element="f.D2E048430D79195F3054D6CB1F377AFB.d.C3CE2A2E765A6A89FC277812E20486A1.e.84")>,
 <selenium.webdriver.remote.webelement.WebElement (session="99d230b05e3d95c53ee3771b7e76fad8", element="f.D2E048430D79195F3054D6CB1F377AFB.d.C3CE2A2E765A6A89FC277812E20486A1.e.88")>,
 <selenium.webdriver.remote.webelement.WebElement (session="99d230b05e3d95c53ee377